# Make plots summarizing outlook, pph, and storm report data

In [3]:
import numpy as np
import geopandas as gp
import shapely as sp
import matplotlib.pyplot as plt
import contextily as cx
import cartopy as cp
from datetime import datetime as dt
import datetime
from datetime import timedelta
import xarray as xr
import pandas as pd
import re
import os
import cartopy.crs as ccrs
from utils_filter import *
from utils_datetime import *
from utils_plotting import *

ModuleNotFoundError: No module named 'utils.utils_filter'

In [8]:
data_location = 'data'
moderate = True # True if only doing days for which there was a moderate outlook

## Read in Datasets

In [10]:
outlooks, pph, reports = read_datasets(data_location, moderate)

reading outlooks
reading pph
reading storm reports


## Plots: Pick some specific days and plot daily data for each datatype


In [32]:
date = datetime.date(2023, 3, 31) # date to make plots for

outlooks_date = select_days_outlooks(outlooks, [date])
pph_date = select_days_pph(pph, [date])
reports_date = select_days_reports(reports, [date])
reports_date = add_significant_column(reports_date)


C:\Users\miles\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\geopandas\geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [33]:
day_output_location = 'plots/daily/' + date.strftime('%Y%m%d')
day_outlook_output_location = day_output_location + '/outlooks'
day_pph_output_location = day_output_location + '/pph'
day_report_output_location = day_output_location + '/reports'

if not os.path.exists(day_output_location):
    os.mkdir(day_output_location)
    os.mkdir(day_outlook_output_location)
    os.mkdir(day_pph_output_location)
    os.mkdir(day_report_output_location)

categories = ['CATEGORICAL', 'TORNADO', 'WIND', 'HAIL']
plot_outlooks_day(outlooks_date, day_outlook_output_location, categories)

if date.year <= 2019:
    pph_categories = ['wind', 'hail', 'tor']
    plot_pph_day(pph_date, day_pph_output_location, pph_categories)

report_categories = ['Hail', 'Thunderstorm Wind', 'Tornado']
plot_reports(reports_date, day_report_output_location, report_categories)

C:\Users\miles\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\geopandas\geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\miles\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\geopandas\geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\miles

## Map of total occurances of each datatype, broken down by: season, hazard type, and ramp up/down/constant, geographical region.
Also try finding centroids. After that would be to compare outlooks and pph with spatial statistics (look at victor's pph paper and textbook from Alex--Forecast Verification: A Practitioners Guide in ATMOS). And try looking at papers that used pph

Also TODO: make git repository

In [39]:
all_report_output_location = 'plots/summary'
reports = add_significant_column(reports)
plot_reports(reports, all_report_output_location, report_categories)

In [70]:
# create empty dataset in with same x and y coords as reports, plus lat an lon as variables, plus hazard type/threshold as a coordinate (try just copying reports dataset)
hazard_types = ['hail', 'wind', 'tor', 'categorical']
thresholds_wind_hail = ['0.05', '0.15', '0.25', '0.35', '0.45', 'SIGN', '0.30', '0.60']
thresholds_tor = ['0.02', '0.05', '0.15', 'SIGN', '0.25', '0.35', '0.10', '0.30', '0.45', '0.60']
thresholds_categorical = ['TSTM', 'MRGL', 'SLGT', 'ENH', 'MDT', 'HIGH']

def create_hazards(hazard_types, thresholds_wind_hail, thresholds_tor, thresholds_categorical):
    list = []
    for hazard in hazard_types:
        if hazard == 'tor':
            for threshold in thresholds_tor:
                list.append(hazard + '_' + threshold)
        elif hazard == 'categorical':
            for threshold in thresholds_categorical:
                list.append(hazard + '_' + threshold)
        else:
            for threshold in thresholds_wind_hail:
                list.append(hazard + '_' + threshold)
    return list

hazards = create_hazards(hazard_types, thresholds_wind_hail, thresholds_tor, thresholds_categorical)

outlooks_map = xr.Dataset(
    data_vars=dict(
        days = 0,
        lon=(['y', 'x'], pph['lon'].data),
        lat=(['y', 'x'], pph['lat'].data),
    ),
    coords=dict(
        x=(['x'], pph.x.data),
        y=(['y'], pph.y.data),
        hazard=hazards
    ),
    attrs=dict(description="number of outlooks of each threshold by grid point.",
               grid = pph.grid),
)

outlooks_map
# create variables for each of 3 hazard types and categorical, for each grid point counting number of times that grid point is in a polygon of outlooks (filtered appropriately)

<xarray.Dataset>
Dimensions:  (y: 65, x: 93, hazard: 32)
Coordinates:
  * x        (x) float64 0.0 1.0 2.0 3.0 4.0 5.0 ... 88.0 89.0 90.0 91.0 92.0
  * y        (y) float64 0.0 1.0 2.0 3.0 4.0 5.0 ... 60.0 61.0 62.0 63.0 64.0
  * hazard   (hazard) <U16 'hail_0.05' 'hail_0.15' ... 'categorical_HIGH'
Data variables:
    days     int32 0
    lon      (y, x) float64 -133.5 -132.8 -132.1 -131.3 ... -51.51 -50.44 -49.39
    lat      (y, x) float64 12.19 12.39 12.58 12.77 ... 57.87 57.68 57.49 57.29
Attributes:
    description:  number of outlooks of each threshold by grid point.
    grid:         80-km NCEP 211

In [59]:
pph

<xarray.Dataset>
Dimensions:             (time: 1515, x: 93, y: 65)
Coordinates:
  * time                (time) datetime64[ns] 1987-02-14T12:00:00 ... 2019-12...
  * x                   (x) float64 0.0 1.0 2.0 3.0 4.0 ... 89.0 90.0 91.0 92.0
  * y                   (y) float64 0.0 1.0 2.0 3.0 4.0 ... 61.0 62.0 63.0 64.0
Data variables:
    lat                 (y, x) float64 ...
    lon                 (y, x) float64 ...
    p_perfect_wind      (time, y, x) float64 ...
    p_perfect_sig_wind  (time, y, x) float64 ...
    p_perfect_hail      (time, y, x) float64 ...
    p_perfect_sig_hail  (time, y, x) float64 ...
    p_perfect_tor       (time, y, x) float64 ...
    p_perfect_sig_tor   (time, y, x) float64 ...
Attributes:
    title:         Practically Perfect Wind Hindcasts
    sigma:         1.5
    grid:          80-km NCEP 211
    author:        Dr. Victor Gensini
    author_email:  vgensini@niu.edu
    citation:      https://doi.org/10.1175/BAMS-D-19-0321.1

In [45]:
outlooks['THRESHOLD'].unique()

array([None, 'SLGT', 'MDT', 'TSTM', 'HIGH', '0.05', '0.15', '0.25',
       '0.02', '0.35', '0.45', 'SIGN', '0.30', '0.10', '0.60', 'MRGL',
       'ENH'], dtype=object)

In [53]:
outlooks[outlooks['CATEGORY'] == 'TORNADO']['THRESHOLD'].unique()

array(['0.02', '0.05', '0.15', 'SIGN', '0.25', '0.35', '0.10', '0.30',
       '0.45', '0.60'], dtype=object)